Instalando Pacote

Puxando Dataset de Discurso de Ódio

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
import pandas as pd
# import transformers
# from transformers import AutoModel, BertTokenizerFast
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from transformers import AdamW
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer



# link: https://huggingface.co/datasets/ag_news

In [3]:
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/t5_key_text/original/base_ironic_test.csv', sep = ';', decimal = ',')
test.head()

,text,label
0,@user Can U Help?||More conservatives needed o...,0
1,"Just walked in to #Starbucks and asked for a ""...",1
2,#NOT GONNA WIN,0
3,@user He is exactly that sort of person. Weirdo!,0
4,So much #sarcasm at work mate 10/10 #boring 10...,1


In [4]:
# X_train = dataset['train']['text']
X_test = test['text']
# y_train = dataset['train']['label']
y_test = test['label']

In [5]:
nclasses = len(np.unique(y_test))

In [6]:
nclasses

2

In [7]:
def puxa_dados_normal(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y


In [8]:
def puxa_original(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')

  X = csv['texto'].values
  y = csv['label'].values


  return X, y

In [9]:
def puxa_dados_normal_pkl(caminho):
  csv = pd.read_pickle(caminho)
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'label']].copy()
  # parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y


In [10]:
def puxa_dados_lambada(caminho, original):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = pd.read_csv(original, sep = ';', decimal = ',')

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'new_label']].copy()
  # parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto", "new_label": "label"}, inplace = True)

  fim = original.append(parafrase)
  # fim['texto'] = fim['texto'].astype('str')
  # fim = fim[fim['label'].isin([0,1])].copy()
  print(fim['label'].unique())

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y

In [11]:
def desempenho_LSTM(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  mask_zero= True,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  input_shape=(max_length, )),

        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [12]:
def desempenho_CNN(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  mask_zero= True,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  input_shape=(max_length, )),

        tf.keras.layers.Conv1D(filters=100, kernel_size = 3, activation = 'relu',
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),

        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu',
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [13]:

y_test = np.array(y_test)


# Original

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_original(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30
5/5 - 2s - loss: 0.7033 - accuracy: 0.5139 - 2s/epoch - 488ms/step
Epoch 2/30
5/5 - 1s - loss: 0.6848 - accuracy: 0.5903 - 563ms/epoch - 113ms/step
Epoch 3/30
5/5 - 0s - loss: 0.6698 - accuracy: 0.6250 - 320ms/epoch - 64ms/step
Epoch 4/30
5/5 - 0s - loss: 0.6572 - accuracy: 0.6111 - 337ms/epoch - 67ms/step
Epoch 5/30
5/5 - 0s - loss: 0.6223 - accuracy: 0.6875 - 324ms/epoch - 65ms/step
Epoch 6/30
5/5 - 0s - loss: 0.6018 - accuracy: 0.6528 - 326ms/epoch - 65ms/step
Epoch 7/30
5/5 - 0s - loss: 0.5697 - accuracy: 0.7222 - 324ms/epoch - 65ms/step
Epoch 8/30
5/5 - 0s - loss: 0.5479 - accuracy: 0.7083 - 334ms/epoch - 67ms/step
Epoch 9/30
5/5 - 0s - loss: 0.4898 - accuracy: 0.8403 - 328ms/epoch - 66ms/step
Epoch 10/30
5/5 - 0s - loss: 0.4707 - accuracy: 0.8264 - 328ms/epoch - 66ms/step
Epoch 11/30
5/5 - 0s - loss: 0.3685 - accuracy: 0.8889 - 347ms/epoch - 69ms/step
Epoch 12/30
5/5 - 0s - loss: 0.3407 - accuracy: 0.8611 - 318ms/epoch - 64ms/step
Epoch 13/30
5/5 - 0s - loss: 0.2814 

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.507653,0.507650,0.547194,0.513262
1,original2,0.399235,0.288748,0.585459,0.582409
2,original3,0.452806,0.452762,0.554847,0.538166
3,original4,0.415816,0.363889,0.534439,0.532613
4,original5,0.544643,0.543626,0.565051,0.555487


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.464031,0.431335,0.557398,0.544387
std,0.061340,0.104609,0.019260,0.026053
min,0.399235,0.288748,0.534439,0.513262
25%,0.415816,0.363889,0.547194,0.532613
50%,0.452806,0.452762,0.554847,0.538166
75%,0.507653,0.507650,0.565051,0.555487
max,0.544643,0.543626,0.585459,0.582409


# EDA

In [14]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/eda/base_ironic_'+str(perc)+'_eda'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/eda/base_ironic_'+str(perc)+'_eda'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/eda/base_ironic_'+str(perc)+'_eda'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/eda/base_ironic_'+str(perc)+'_eda'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/eda/base_ironic_'+str(perc)+'_eda'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
23/23 - 15s - loss: 0.6921 - accuracy: 0.5285 - 15s/epoch - 656ms/step
Epoch 2/30
23/23 - 4s - loss: 0.6638 - accuracy: 0.6050 - 4s/epoch - 156ms/step
Epoch 3/30
23/23 - 4s - loss: 0.5831 - accuracy: 0.6759 - 4s/epoch - 164ms/step
Epoch 4/30
23/23 - 2s - loss: 0.4387 - accuracy: 0.7232 - 2s/epoch - 88ms/step
Epoch 5/30
23/23 - 2s - loss: 0.3208 - accuracy: 0.7399 - 2s/epoch - 94ms/step
Epoch 6/30
23/23 - 1s - loss: 0.2729 - accuracy: 0.8442 - 1s/epoch - 60ms/step
Epoch 7/30
23/23 - 1s - loss: 0.2838 - accuracy: 0.8623 - 1s/epoch - 58ms/step
Epoch 8/30
23/23 - 1s - loss: 0.2005 - accuracy: 0.9152 - 950ms/epoch - 41ms/step
Epoch 9/30
23/23 - 1s - loss: 0.1560 - accuracy: 0.9193 - 1s/epoch - 44ms/step
Epoch 10/30
23/23 - 0s - loss: 0.1243 - accuracy: 0.9485 - 122ms/epoch - 5ms/step
Epoch 11/30
23/23 - 1s - loss: 0.1173 - accuracy: 0.9444 - 936ms/epoch - 41ms/step
Epoch 12/30
23/23 - 1s - loss: 0.1127 - accuracy: 0.9402 - 816ms/epoch - 35ms/step
Epoch 13/30
23/23 - 1s - loss: 0.

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


23/23 - 6s - loss: 0.6914 - accuracy: 0.5167 - 6s/epoch - 243ms/step
Epoch 2/30
23/23 - 4s - loss: 0.6708 - accuracy: 0.6125 - 4s/epoch - 184ms/step
Epoch 3/30
23/23 - 3s - loss: 0.6083 - accuracy: 0.7139 - 3s/epoch - 129ms/step
Epoch 4/30
23/23 - 2s - loss: 0.4303 - accuracy: 0.8500 - 2s/epoch - 95ms/step
Epoch 5/30
23/23 - 1s - loss: 0.2120 - accuracy: 0.9458 - 1s/epoch - 60ms/step
Epoch 6/30
23/23 - 2s - loss: 0.1412 - accuracy: 0.9625 - 2s/epoch - 67ms/step
Epoch 7/30
23/23 - 2s - loss: 0.1026 - accuracy: 0.9694 - 2s/epoch - 86ms/step
Epoch 8/30
23/23 - 1s - loss: 0.0721 - accuracy: 0.9708 - 1s/epoch - 59ms/step
Epoch 9/30
23/23 - 1s - loss: 0.0691 - accuracy: 0.9639 - 1s/epoch - 46ms/step
Epoch 10/30
23/23 - 1s - loss: 0.0632 - accuracy: 0.9611 - 841ms/epoch - 37ms/step
Epoch 11/30
23/23 - 1s - loss: 0.0713 - accuracy: 0.9611 - 814ms/epoch - 35ms/step
Epoch 12/30
23/23 - 1s - loss: 0.0742 - accuracy: 0.9611 - 1s/epoch - 64ms/step
Epoch 13/30
23/23 - 1s - loss: 0.0573 - accuracy: 0

...

23/23 - 5s - loss: 0.6932 - accuracy: 0.5264 - 5s/epoch - 224ms/step
Epoch 2/30
23/23 - 4s - loss: 0.6589 - accuracy: 0.6514 - 4s/epoch - 189ms/step
Epoch 3/30
23/23 - 4s - loss: 0.5417 - accuracy: 0.7583 - 4s/epoch - 157ms/step
Epoch 4/30
23/23 - 2s - loss: 0.3277 - accuracy: 0.8319 - 2s/epoch - 94ms/step
Epoch 5/30
23/23 - 1s - loss: 0.1679 - accuracy: 0.9361 - 928ms/epoch - 40ms/step
Epoch 6/30
23/23 - 2s - loss: 0.1306 - accuracy: 0.9500 - 2s/epoch - 87ms/step
Epoch 7/30
23/23 - 1s - loss: 0.1201 - accuracy: 0.9639 - 1s/epoch - 58ms/step
Epoch 8/30
23/23 - 1s - loss: 0.1194 - accuracy: 0.9625 - 536ms/epoch - 23ms/step
Epoch 9/30
23/23 - 1s - loss: 0.0946 - accuracy: 0.9847 - 727ms/epoch - 32ms/step
Epoch 10/30
23/23 - 1s - loss: 0.0894 - accuracy: 0.9875 - 744ms/epoch - 32ms/step
Epoch 11/30
23/23 - 1s - loss: 0.0921 - accuracy: 0.9917 - 535ms/epoch - 23ms/step
Epoch 12/30
23/23 - 1s - loss: 0.1057 - accuracy: 0.9819 - 507ms/epoch - 22ms/step
Epoch 13/30
23/23 - 1s - loss: 0.1024 -

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


23/23 - 6s - loss: 0.6924 - accuracy: 0.5049 - 6s/epoch - 245ms/step
Epoch 2/30
23/23 - 4s - loss: 0.6636 - accuracy: 0.5981 - 4s/epoch - 170ms/step
Epoch 3/30
23/23 - 3s - loss: 0.6097 - accuracy: 0.6453 - 3s/epoch - 136ms/step
Epoch 4/30
23/23 - 2s - loss: 0.3995 - accuracy: 0.7844 - 2s/epoch - 102ms/step
Epoch 5/30
23/23 - 2s - loss: 0.2475 - accuracy: 0.8303 - 2s/epoch - 94ms/step
Epoch 6/30
23/23 - 1s - loss: 0.1974 - accuracy: 0.8846 - 1s/epoch - 58ms/step
Epoch 7/30
23/23 - 2s - loss: 0.1631 - accuracy: 0.9054 - 2s/epoch - 84ms/step
Epoch 8/30
23/23 - 1s - loss: 0.1297 - accuracy: 0.9583 - 1s/epoch - 53ms/step
Epoch 9/30
23/23 - 1s - loss: 0.1062 - accuracy: 0.9541 - 1s/epoch - 51ms/step
Epoch 10/30
23/23 - 1s - loss: 0.1087 - accuracy: 0.9666 - 509ms/epoch - 22ms/step
Epoch 11/30
23/23 - 1s - loss: 0.0836 - accuracy: 0.9750 - 501ms/epoch - 22ms/step
Epoch 12/30
23/23 - 1s - loss: 0.0694 - accuracy: 0.9694 - 1s/epoch - 49ms/step
Epoch 13/30
23/23 - 1s - loss: 0.0649 - accuracy: 

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


23/23 - 6s - loss: 0.6862 - accuracy: 0.5375 - 6s/epoch - 264ms/step
Epoch 2/30
23/23 - 4s - loss: 0.6584 - accuracy: 0.6250 - 4s/epoch - 162ms/step
Epoch 3/30
23/23 - 3s - loss: 0.5409 - accuracy: 0.7625 - 3s/epoch - 145ms/step
Epoch 4/30
23/23 - 2s - loss: 0.2945 - accuracy: 0.8986 - 2s/epoch - 77ms/step
Epoch 5/30
23/23 - 2s - loss: 0.1463 - accuracy: 0.9264 - 2s/epoch - 86ms/step
Epoch 6/30
23/23 - 1s - loss: 0.1063 - accuracy: 0.9264 - 1s/epoch - 50ms/step
Epoch 7/30
23/23 - 1s - loss: 0.0995 - accuracy: 0.9403 - 1s/epoch - 50ms/step
Epoch 8/30
23/23 - 1s - loss: 0.0896 - accuracy: 0.9431 - 718ms/epoch - 31ms/step
Epoch 9/30
23/23 - 1s - loss: 0.0984 - accuracy: 0.9236 - 744ms/epoch - 32ms/step
Epoch 10/30
23/23 - 1s - loss: 0.0756 - accuracy: 0.9514 - 1s/epoch - 64ms/step
Epoch 11/30
23/23 - 1s - loss: 0.0924 - accuracy: 0.9347 - 1s/epoch - 49ms/step
Epoch 12/30
23/23 - 1s - loss: 0.0790 - accuracy: 0.9458 - 873ms/epoch - 38ms/step
Epoch 13/30
23/23 - 1s - loss: 0.0761 - accuracy

In [15]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [16]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.519133,0.513413,0.478316,0.477152
1,original2,0.506378,0.506087,0.442602,0.375495
2,original3,0.483418,0.477563,0.480867,0.474090
3,original4,0.500000,0.491731,0.543367,0.525041
4,original5,0.554847,0.547927,0.544643,0.543269


In [17]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.512755,0.507344,0.497959,0.479009
std,0.026816,0.026533,0.044676,0.065175
min,0.483418,0.477563,0.442602,0.375495
25%,0.500000,0.491731,0.478316,0.474090
50%,0.506378,0.506087,0.480867,0.477152
75%,0.519133,0.513413,0.543367,0.525041
max,0.554847,0.547927,0.544643,0.543269


# SSMBA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/ssmba/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/ssmba/base_ironic_'+str(perc)+'_ssmba'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/ssmba/base_ironic_'+str(perc)+'_ssmba'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/ssmba/base_ironic_'+str(perc)+'_ssmba'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/ssmba/base_ironic_'+str(perc)+'_ssmba'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 6s - loss: 0.6891 - accuracy: 0.5081 - 6s/epoch - 223ms/step
Epoch 2/30
27/27 - 6s - loss: 0.6571 - accuracy: 0.6366 - 6s/epoch - 214ms/step
Epoch 3/30
27/27 - 3s - loss: 0.5348 - accuracy: 0.7766 - 3s/epoch - 116ms/step
Epoch 4/30
27/27 - 3s - loss: 0.2684 - accuracy: 0.9317 - 3s/epoch - 106ms/step
Epoch 5/30
27/27 - 1s - loss: 0.1474 - accuracy: 0.9572 - 1s/epoch - 54ms/step
Epoch 6/30
27/27 - 1s - loss: 0.1324 - accuracy: 0.9572 - 1s/epoch - 55ms/step
Epoch 7/30
27/27 - 2s - loss: 0.1006 - accuracy: 0.9630 - 2s/epoch - 60ms/step
Epoch 8/30
27/27 - 2s - loss: 0.0879 - accuracy: 0.9664 - 2s/epoch - 61ms/step
Epoch 9/30
27/27 - 1s - loss: 0.1279 - accuracy: 0.9664 - 532ms/epoch - 20ms/step
Epoch 10/30
27/27 - 1s - loss: 0.0969 - accuracy: 0.9688 - 535ms/epoch - 20ms/step
Epoch 11/30
27/27 - 1s - loss: 0.0905 - accuracy: 0.9699 - 520ms/epoch - 19ms/step
Epoch 12/30
27/27 - 0s - loss: 0.0936 - accuracy: 0.9734 - 394ms/epoch - 15ms/step
Epoch 13/30
27/27 - 0s - loss: 0.1031 - accu

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 9s - loss: 0.6958 - accuracy: 0.5233 - 9s/epoch - 315ms/step
Epoch 2/30
27/27 - 3s - loss: 0.6753 - accuracy: 0.5618 - 3s/epoch - 95ms/step
Epoch 3/30
27/27 - 3s - loss: 0.6293 - accuracy: 0.6154 - 3s/epoch - 127ms/step
Epoch 4/30
27/27 - 2s - loss: 0.4886 - accuracy: 0.8042 - 2s/epoch - 79ms/step
Epoch 5/30
27/27 - 3s - loss: 0.1958 - accuracy: 0.9149 - 3s/epoch - 111ms/step
Epoch 6/30
27/27 - 1s - loss: 0.0905 - accuracy: 0.9382 - 919ms/epoch - 34ms/step
Epoch 7/30
27/27 - 2s - loss: 0.0800 - accuracy: 0.9312 - 2s/epoch - 68ms/step
Epoch 8/30
27/27 - 1s - loss: 0.0719 - accuracy: 0.9452 - 578ms/epoch - 21ms/step
Epoch 9/30
27/27 - 1s - loss: 0.0680 - accuracy: 0.9441 - 791ms/epoch - 29ms/step
Epoch 10/30
27/27 - 1s - loss: 0.0535 - accuracy: 0.9557 - 581ms/epoch - 22ms/step
Epoch 11/30
27/27 - 1s - loss: 0.0705 - accuracy: 0.9336 - 573ms/epoch - 21ms/step
Epoch 12/30
27/27 - 1s - loss: 0.0522 - accuracy: 0.9441 - 806ms/epoch - 30ms/step
Epoch 13/30
27/27 - 1s - loss: 0.0515 -

...

27/27 - 7s - loss: 0.6940 - accuracy: 0.5440 - 7s/epoch - 274ms/step
Epoch 2/30
27/27 - 4s - loss: 0.6701 - accuracy: 0.6204 - 4s/epoch - 137ms/step
Epoch 3/30
27/27 - 1s - loss: 0.5386 - accuracy: 0.7824 - 1s/epoch - 46ms/step
Epoch 4/30
27/27 - 1s - loss: 0.2911 - accuracy: 0.9109 - 1s/epoch - 55ms/step
Epoch 5/30
27/27 - 2s - loss: 0.1564 - accuracy: 0.9433 - 2s/epoch - 82ms/step
Epoch 6/30
27/27 - 1s - loss: 0.1355 - accuracy: 0.9549 - 1s/epoch - 43ms/step
Epoch 7/30
27/27 - 2s - loss: 0.1115 - accuracy: 0.9618 - 2s/epoch - 87ms/step
Epoch 8/30
Epoch 9/30
27/27 - 1s - loss: 0.0974 - accuracy: 0.9838 - 1s/epoch - 38ms/step
Epoch 10/30
27/27 - 1s - loss: 0.0838 - accuracy: 0.9919 - 781ms/epoch - 29ms/step
Epoch 11/30
27/27 - 1s - loss: 0.0506 - accuracy: 0.9942 - 780ms/epoch - 29ms/step
Epoch 12/30
27/27 - 1s - loss: 0.0525 - accuracy: 0.9931 - 787ms/epoch - 29ms/step
Epoch 13/30
27/27 - 1s - loss: 0.0453 - accuracy: 0.9954 - 568ms/epoch - 21ms/step
Epoch 14/30
27/27 - 0s - loss: 0.0

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 8s - loss: 0.6977 - accuracy: 0.4780 - 8s/epoch - 300ms/step
Epoch 2/30
27/27 - 4s - loss: 0.6835 - accuracy: 0.5243 - 4s/epoch - 145ms/step
Epoch 3/30
27/27 - 1s - loss: 0.6345 - accuracy: 0.6123 - 790ms/epoch - 29ms/step
Epoch 4/30
27/27 - 2s - loss: 0.4666 - accuracy: 0.8137 - 2s/epoch - 87ms/step
Epoch 5/30
27/27 - 1s - loss: 0.2453 - accuracy: 0.9155 - 1s/epoch - 46ms/step
Epoch 6/30
27/27 - 1s - loss: 0.1328 - accuracy: 0.9699 - 1s/epoch - 46ms/step
Epoch 7/30
27/27 - 2s - loss: 0.0944 - accuracy: 0.9618 - 2s/epoch - 63ms/step
Epoch 8/30
27/27 - 1s - loss: 0.0785 - accuracy: 0.9688 - 1s/epoch - 47ms/step
Epoch 9/30
27/27 - 2s - loss: 0.0745 - accuracy: 0.9641 - 2s/epoch - 61ms/step
Epoch 10/30
27/27 - 2s - loss: 0.0633 - accuracy: 0.9664 - 2s/epoch - 58ms/step
Epoch 11/30
27/27 - 1s - loss: 0.0739 - accuracy: 0.9664 - 585ms/epoch - 22ms/step
Epoch 12/30
27/27 - 1s - loss: 0.0644 - accuracy: 0.9630 - 576ms/epoch - 21ms/step
Epoch 13/30
27/27 - 1s - loss: 0.0618 - accuracy:

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 6s - loss: 0.6963 - accuracy: 0.4942 - 6s/epoch - 231ms/step
Epoch 2/30
27/27 - 6s - loss: 0.6883 - accuracy: 0.5233 - 6s/epoch - 217ms/step
Epoch 3/30
27/27 - 3s - loss: 0.6787 - accuracy: 0.5618 - 3s/epoch - 104ms/step
Epoch 4/30
27/27 - 2s - loss: 0.5402 - accuracy: 0.7284 - 2s/epoch - 70ms/step
Epoch 5/30
27/27 - 1s - loss: 0.1865 - accuracy: 0.9522 - 1s/epoch - 47ms/step
Epoch 6/30
27/27 - 1s - loss: 0.0783 - accuracy: 0.9825 - 1s/epoch - 46ms/step
Epoch 7/30
27/27 - 1s - loss: 0.0657 - accuracy: 0.9883 - 803ms/epoch - 30ms/step
Epoch 8/30
27/27 - 0s - loss: 0.0522 - accuracy: 0.9953 - 357ms/epoch - 13ms/step
Epoch 9/30
27/27 - 2s - loss: 0.0438 - accuracy: 0.9942 - 2s/epoch - 69ms/step
Epoch 10/30
27/27 - 1s - loss: 0.0497 - accuracy: 0.9918 - 533ms/epoch - 20ms/step
Epoch 11/30
27/27 - 1s - loss: 0.0431 - accuracy: 0.9942 - 916ms/epoch - 34ms/step
Epoch 12/30
27/27 - 1s - loss: 0.0418 - accuracy: 0.9965 - 1s/epoch - 47ms/step
Epoch 13/30
27/27 - 0s - loss: 0.0442 - accur

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.496173,0.496166,0.529337,0.486269
1,original2,0.471939,0.468828,0.529337,0.529115
2,original3,0.538265,0.519230,0.496173,0.488010
3,original4,0.496173,0.494508,0.528061,0.522313
4,original5,0.487245,0.472285,0.522959,0.520513


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.497959,0.490204,0.521173,0.509244
std,0.024611,0.020461,0.014221,0.020442
min,0.471939,0.468828,0.496173,0.486269
25%,0.487245,0.472285,0.522959,0.488010
50%,0.496173,0.494508,0.528061,0.520513
75%,0.496173,0.496166,0.529337,0.522313
max,0.538265,0.519230,0.529337,0.529115


# Checklist

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/checklist/base_ironic_'+str(perc)+'_checklist'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/checklist/base_ironic_'+str(perc)+'_checklist'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/checklist/base_ironic_'+str(perc)+'_checklist'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/checklist/base_ironic_'+str(perc)+'_checklist'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/checklist/base_ironic_'+str(perc)+'_checklist'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


16/16 - 5s - loss: 0.6945 - accuracy: 0.5041 - 5s/epoch - 297ms/step
Epoch 2/30
16/16 - 4s - loss: 0.6678 - accuracy: 0.6058 - 4s/epoch - 240ms/step
Epoch 3/30
16/16 - 2s - loss: 0.6395 - accuracy: 0.6224 - 2s/epoch - 147ms/step
Epoch 4/30
16/16 - 2s - loss: 0.5335 - accuracy: 0.7925 - 2s/epoch - 123ms/step
Epoch 5/30
16/16 - 2s - loss: 0.3733 - accuracy: 0.9004 - 2s/epoch - 123ms/step
Epoch 6/30
16/16 - 2s - loss: 0.2355 - accuracy: 0.9523 - 2s/epoch - 109ms/step
Epoch 7/30
16/16 - 2s - loss: 0.1534 - accuracy: 0.9585 - 2s/epoch - 103ms/step
Epoch 8/30
16/16 - 2s - loss: 0.1409 - accuracy: 0.9481 - 2s/epoch - 113ms/step
Epoch 9/30
16/16 - 1s - loss: 0.1172 - accuracy: 0.9585 - 1s/epoch - 69ms/step
Epoch 10/30
16/16 - 1s - loss: 0.1121 - accuracy: 0.9627 - 710ms/epoch - 44ms/step
Epoch 11/30
16/16 - 1s - loss: 0.1056 - accuracy: 0.9772 - 740ms/epoch - 46ms/step
Epoch 12/30
16/16 - 1s - loss: 0.1196 - accuracy: 0.9689 - 1s/epoch - 83ms/step
Epoch 13/30
16/16 - 0s - loss: 0.0838 - accura

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


16/16 - 4s - loss: 0.6942 - accuracy: 0.4623 - 4s/epoch - 267ms/step
Epoch 2/30
16/16 - 2s - loss: 0.6916 - accuracy: 0.5255 - 2s/epoch - 123ms/step
Epoch 3/30
16/16 - 2s - loss: 0.6862 - accuracy: 0.5214 - 2s/epoch - 99ms/step
Epoch 4/30
16/16 - 2s - loss: 0.6735 - accuracy: 0.5438 - 2s/epoch - 135ms/step
Epoch 5/30
16/16 - 1s - loss: 0.6406 - accuracy: 0.6049 - 1s/epoch - 71ms/step
Epoch 6/30
16/16 - 2s - loss: 0.5289 - accuracy: 0.7006 - 2s/epoch - 102ms/step
Epoch 7/30
16/16 - 2s - loss: 0.3402 - accuracy: 0.8513 - 2s/epoch - 109ms/step
Epoch 8/30
16/16 - 1s - loss: 0.2722 - accuracy: 0.8534 - 850ms/epoch - 53ms/step
Epoch 9/30
16/16 - 1s - loss: 0.2685 - accuracy: 0.8595 - 1s/epoch - 76ms/step
Epoch 10/30
16/16 - 1s - loss: 0.2726 - accuracy: 0.8493 - 863ms/epoch - 54ms/step
Epoch 11/30
16/16 - 1s - loss: 0.2810 - accuracy: 0.8574 - 1s/epoch - 64ms/step
Epoch 12/30
16/16 - 1s - loss: 0.2692 - accuracy: 0.8432 - 646ms/epoch - 40ms/step
Epoch 13/30
16/16 - 1s - loss: 0.2387 - accura

...

16/16 - 5s - loss: 0.6930 - accuracy: 0.4724 - 5s/epoch - 296ms/step
Epoch 2/30
16/16 - 2s - loss: 0.6848 - accuracy: 0.5194 - 2s/epoch - 151ms/step
Epoch 3/30
16/16 - 4s - loss: 0.6573 - accuracy: 0.5644 - 4s/epoch - 240ms/step
Epoch 4/30
16/16 - 2s - loss: 0.5858 - accuracy: 0.6339 - 2s/epoch - 141ms/step
Epoch 5/30
16/16 - 2s - loss: 0.4851 - accuracy: 0.6933 - 2s/epoch - 114ms/step
Epoch 6/30
16/16 - 2s - loss: 0.3555 - accuracy: 0.8139 - 2s/epoch - 98ms/step
Epoch 7/30
16/16 - 1s - loss: 0.2009 - accuracy: 0.8998 - 1s/epoch - 71ms/step
Epoch 8/30
16/16 - 1s - loss: 0.1120 - accuracy: 0.9264 - 720ms/epoch - 45ms/step
Epoch 9/30
16/16 - 1s - loss: 0.0679 - accuracy: 0.9530 - 1s/epoch - 71ms/step
Epoch 10/30
16/16 - 0s - loss: 0.0639 - accuracy: 0.9387 - 308ms/epoch - 19ms/step
Epoch 11/30
16/16 - 1s - loss: 0.0685 - accuracy: 0.9387 - 505ms/epoch - 32ms/step
Epoch 12/30
16/16 - 1s - loss: 0.0574 - accuracy: 0.9775 - 931ms/epoch - 58ms/step
Epoch 13/30
16/16 - 1s - loss: 0.0613 - acc

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


15/15 - 6s - loss: 0.6825 - accuracy: 0.5496 - 6s/epoch - 410ms/step
Epoch 2/30
15/15 - 3s - loss: 0.6016 - accuracy: 0.7500 - 3s/epoch - 188ms/step
Epoch 3/30
15/15 - 2s - loss: 0.4530 - accuracy: 0.8060 - 2s/epoch - 130ms/step
Epoch 4/30
15/15 - 2s - loss: 0.3417 - accuracy: 0.8599 - 2s/epoch - 145ms/step
Epoch 5/30
15/15 - 1s - loss: 0.2087 - accuracy: 0.9138 - 904ms/epoch - 60ms/step
Epoch 6/30
15/15 - 1s - loss: 0.1403 - accuracy: 0.9224 - 1s/epoch - 90ms/step
Epoch 7/30
15/15 - 1s - loss: 0.1206 - accuracy: 0.9332 - 713ms/epoch - 48ms/step
Epoch 8/30
15/15 - 0s - loss: 0.1149 - accuracy: 0.9353 - 428ms/epoch - 29ms/step
Epoch 9/30
15/15 - 2s - loss: 0.0946 - accuracy: 0.9526 - 2s/epoch - 123ms/step
Epoch 10/30
15/15 - 2s - loss: 0.0926 - accuracy: 0.9397 - 2s/epoch - 116ms/step
Epoch 11/30
15/15 - 1s - loss: 0.0961 - accuracy: 0.9353 - 503ms/epoch - 34ms/step
Epoch 12/30
15/15 - 1s - loss: 0.0865 - accuracy: 0.9569 - 910ms/epoch - 61ms/step
Epoch 13/30
15/15 - 1s - loss: 0.0946 -

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


14/14 - 5s - loss: 0.6942 - accuracy: 0.5035 - 5s/epoch - 359ms/step
Epoch 2/30
14/14 - 2s - loss: 0.6735 - accuracy: 0.5130 - 2s/epoch - 154ms/step
Epoch 3/30
14/14 - 2s - loss: 0.6181 - accuracy: 0.6217 - 2s/epoch - 169ms/step
Epoch 4/30
14/14 - 1s - loss: 0.5506 - accuracy: 0.7423 - 1s/epoch - 73ms/step
Epoch 5/30
14/14 - 2s - loss: 0.4706 - accuracy: 0.8298 - 2s/epoch - 113ms/step
Epoch 6/30
14/14 - 1s - loss: 0.3135 - accuracy: 0.9007 - 1s/epoch - 87ms/step
Epoch 7/30
14/14 - 1s - loss: 0.2283 - accuracy: 0.9243 - 1s/epoch - 105ms/step
Epoch 8/30
14/14 - 2s - loss: 0.1834 - accuracy: 0.9149 - 2s/epoch - 161ms/step
Epoch 9/30
14/14 - 1s - loss: 0.1509 - accuracy: 0.9220 - 768ms/epoch - 55ms/step
Epoch 10/30
14/14 - 1s - loss: 0.1121 - accuracy: 0.9149 - 846ms/epoch - 60ms/step
Epoch 11/30
14/14 - 0s - loss: 0.0785 - accuracy: 0.9409 - 460ms/epoch - 33ms/step
Epoch 12/30
14/14 - 0s - loss: 0.0816 - accuracy: 0.9314 - 457ms/epoch - 33ms/step
Epoch 13/30
14/14 - 0s - loss: 0.0499 - ac

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.470663,0.467439,0.579082,0.547211
1,original2,0.491071,0.487327,0.557398,0.557138
2,original3,0.446429,0.444984,0.517857,0.495227
3,original4,0.498724,0.494579,0.488520,0.474737
4,original5,0.596939,0.579422,0.581633,0.522448


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.500765,0.494750,0.544898,0.519352
std,0.057459,0.051105,0.040570,0.034590
min,0.446429,0.444984,0.488520,0.474737
25%,0.470663,0.467439,0.517857,0.495227
50%,0.491071,0.487327,0.557398,0.522448
75%,0.498724,0.494579,0.579082,0.547211
max,0.596939,0.579422,0.581633,0.557138


# Back Translation

In [21]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/back_translation/base_ironic_'+str(perc)+'_back_translation'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/back_translation/base_ironic_'+str(perc)+'_back_translation'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/back_translation/base_ironic_'+str(perc)+'_back_translation'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/back_translation/base_ironic_'+str(perc)+'_back_translation'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/back_translation/base_ironic_'+str(perc)+'_back_translation'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


9/9 - 3s - loss: 0.6981 - accuracy: 0.4861 - 3s/epoch - 372ms/step
Epoch 2/30
9/9 - 2s - loss: 0.6863 - accuracy: 0.5486 - 2s/epoch - 194ms/step
Epoch 3/30
9/9 - 2s - loss: 0.6577 - accuracy: 0.6042 - 2s/epoch - 169ms/step
Epoch 4/30
9/9 - 1s - loss: 0.6451 - accuracy: 0.5799 - 1s/epoch - 145ms/step
Epoch 5/30
9/9 - 1s - loss: 0.6040 - accuracy: 0.7083 - 1s/epoch - 118ms/step
Epoch 6/30
9/9 - 2s - loss: 0.5542 - accuracy: 0.7049 - 2s/epoch - 201ms/step
Epoch 7/30
9/9 - 2s - loss: 0.5181 - accuracy: 0.7222 - 2s/epoch - 207ms/step
Epoch 8/30
9/9 - 1s - loss: 0.4412 - accuracy: 0.8160 - 678ms/epoch - 75ms/step
Epoch 9/30
9/9 - 1s - loss: 0.3815 - accuracy: 0.8715 - 882ms/epoch - 98ms/step
Epoch 10/30
9/9 - 1s - loss: 0.2667 - accuracy: 0.9028 - 687ms/epoch - 76ms/step
Epoch 11/30
9/9 - 0s - loss: 0.2311 - accuracy: 0.9236 - 460ms/epoch - 51ms/step
Epoch 12/30
9/9 - 1s - loss: 0.1696 - accuracy: 0.9375 - 690ms/epoch - 77ms/step
Epoch 13/30
9/9 - 0s - loss: 0.1823 - accuracy: 0.9167 - 269ms

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


9/9 - 5s - loss: 0.6958 - accuracy: 0.5000 - 5s/epoch - 521ms/step
Epoch 2/30
9/9 - 2s - loss: 0.6822 - accuracy: 0.5625 - 2s/epoch - 188ms/step
Epoch 3/30
9/9 - 2s - loss: 0.6628 - accuracy: 0.6354 - 2s/epoch - 188ms/step
Epoch 4/30
9/9 - 1s - loss: 0.6339 - accuracy: 0.6736 - 1s/epoch - 164ms/step
Epoch 5/30
9/9 - 1s - loss: 0.5861 - accuracy: 0.7257 - 1s/epoch - 120ms/step
Epoch 6/30
9/9 - 1s - loss: 0.4915 - accuracy: 0.8090 - 1s/epoch - 119ms/step
Epoch 7/30
9/9 - 0s - loss: 0.3847 - accuracy: 0.8854 - 464ms/epoch - 52ms/step
Epoch 8/30
9/9 - 1s - loss: 0.2771 - accuracy: 0.9340 - 1s/epoch - 120ms/step
Epoch 9/30
9/9 - 1s - loss: 0.1833 - accuracy: 0.9375 - 664ms/epoch - 74ms/step
Epoch 10/30
9/9 - 1s - loss: 0.1055 - accuracy: 0.9618 - 503ms/epoch - 56ms/step
Epoch 11/30
9/9 - 1s - loss: 0.0923 - accuracy: 0.9757 - 1s/epoch - 159ms/step
Epoch 12/30
9/9 - 0s - loss: 0.0750 - accuracy: 0.9618 - 394ms/epoch - 44ms/step
Epoch 13/30
9/9 - 1s - loss: 0.0449 - accuracy: 0.9826 - 1s/epoc

...

9/9 - 3s - loss: 0.6957 - accuracy: 0.4826 - 3s/epoch - 340ms/step
Epoch 2/30
9/9 - 2s - loss: 0.6862 - accuracy: 0.5347 - 2s/epoch - 192ms/step
Epoch 3/30
9/9 - 2s - loss: 0.6657 - accuracy: 0.6146 - 2s/epoch - 261ms/step
Epoch 4/30
9/9 - 2s - loss: 0.6244 - accuracy: 0.6493 - 2s/epoch - 182ms/step
Epoch 5/30
9/9 - 2s - loss: 0.5741 - accuracy: 0.6979 - 2s/epoch - 169ms/step
Epoch 6/30
9/9 - 0s - loss: 0.5216 - accuracy: 0.7812 - 461ms/epoch - 51ms/step
Epoch 7/30
9/9 - 1s - loss: 0.4380 - accuracy: 0.7708 - 858ms/epoch - 95ms/step
Epoch 8/30
9/9 - 1s - loss: 0.3970 - accuracy: 0.7847 - 1s/epoch - 118ms/step
Epoch 9/30
9/9 - 1s - loss: 0.3201 - accuracy: 0.8264 - 649ms/epoch - 72ms/step
Epoch 10/30
9/9 - 0s - loss: 0.2519 - accuracy: 0.8611 - 452ms/epoch - 50ms/step
Epoch 11/30
9/9 - 1s - loss: 0.2152 - accuracy: 0.8819 - 903ms/epoch - 100ms/step
Epoch 12/30
9/9 - 0s - loss: 0.2358 - accuracy: 0.8507 - 251ms/epoch - 28ms/step
Epoch 13/30
9/9 - 1s - loss: 0.1796 - accuracy: 0.8924 - 65

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


9/9 - 4s - loss: 0.6970 - accuracy: 0.4722 - 4s/epoch - 422ms/step
Epoch 2/30
9/9 - 2s - loss: 0.6820 - accuracy: 0.5417 - 2s/epoch - 264ms/step
Epoch 3/30
9/9 - 1s - loss: 0.6591 - accuracy: 0.5938 - 1s/epoch - 121ms/step
Epoch 4/30
9/9 - 1s - loss: 0.6244 - accuracy: 0.6736 - 1s/epoch - 151ms/step
Epoch 5/30
9/9 - 1s - loss: 0.6051 - accuracy: 0.6597 - 1s/epoch - 120ms/step
Epoch 6/30
9/9 - 1s - loss: 0.5384 - accuracy: 0.7535 - 1s/epoch - 123ms/step
Epoch 7/30
9/9 - 1s - loss: 0.4864 - accuracy: 0.7882 - 655ms/epoch - 73ms/step
Epoch 8/30
9/9 - 1s - loss: 0.4095 - accuracy: 0.8229 - 866ms/epoch - 96ms/step
Epoch 9/30
9/9 - 1s - loss: 0.3377 - accuracy: 0.8403 - 867ms/epoch - 96ms/step
Epoch 10/30
9/9 - 1s - loss: 0.2988 - accuracy: 0.8333 - 668ms/epoch - 74ms/step
Epoch 11/30
9/9 - 1s - loss: 0.2695 - accuracy: 0.8333 - 1s/epoch - 120ms/step
Epoch 12/30
9/9 - 1s - loss: 0.1935 - accuracy: 0.8785 - 1s/epoch - 163ms/step
Epoch 13/30
9/9 - 1s - loss: 0.1962 - accuracy: 0.8576 - 1s/epoc

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


9/9 - 4s - loss: 0.7001 - accuracy: 0.5208 - 4s/epoch - 456ms/step
Epoch 2/30
9/9 - 1s - loss: 0.6790 - accuracy: 0.5486 - 1s/epoch - 145ms/step
Epoch 3/30
9/9 - 1s - loss: 0.6383 - accuracy: 0.6562 - 1s/epoch - 142ms/step
Epoch 4/30
9/9 - 1s - loss: 0.6031 - accuracy: 0.6528 - 1s/epoch - 141ms/step
Epoch 5/30
9/9 - 2s - loss: 0.5354 - accuracy: 0.8021 - 2s/epoch - 190ms/step
Epoch 6/30
9/9 - 1s - loss: 0.4511 - accuracy: 0.8507 - 662ms/epoch - 74ms/step
Epoch 7/30
9/9 - 1s - loss: 0.3578 - accuracy: 0.8924 - 1s/epoch - 120ms/step
Epoch 8/30
9/9 - 1s - loss: 0.2825 - accuracy: 0.9132 - 1s/epoch - 121ms/step
Epoch 9/30
9/9 - 0s - loss: 0.2416 - accuracy: 0.9410 - 468ms/epoch - 52ms/step
Epoch 10/30
9/9 - 2s - loss: 0.1890 - accuracy: 0.9618 - 2s/epoch - 193ms/step
Epoch 11/30
9/9 - 1s - loss: 0.1651 - accuracy: 0.9722 - 1s/epoch - 122ms/step
Epoch 12/30
9/9 - 0s - loss: 0.1179 - accuracy: 0.9722 - 435ms/epoch - 48ms/step
Epoch 13/30
9/9 - 1s - loss: 0.1244 - accuracy: 0.9757 - 1s/epoch 

In [22]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [23]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.501276,0.496669,0.498724,0.473067
1,original2,0.455357,0.448247,0.551020,0.549602
2,original3,0.498724,0.497940,0.487245,0.485474
3,original4,0.571429,0.524626,0.581633,0.567852
4,original5,0.566327,0.560535,0.573980,0.539683


In [24]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.518622,0.505603,0.538520,0.523136
std,0.049405,0.041254,0.043258,0.041531
min,0.455357,0.448247,0.487245,0.473067
25%,0.498724,0.496669,0.498724,0.485474
50%,0.501276,0.497940,0.551020,0.539683
75%,0.566327,0.524626,0.573980,0.549602
max,0.571429,0.560535,0.581633,0.567852


# LAMBADA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/lambada/base_ironic_'+str(perc)+'_lambada'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/lambada/base_ironic_'+str(perc)+'_lambada'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/lambada/base_ironic_'+str(perc)+'_lambada'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/lambada/base_ironic_'+str(perc)+'_lambada'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/lambada/base_ironic_'+str(perc)+'_lambada'+str(i+5)+'.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-18-0da4cc95c167>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


[0 1]
Epoch 1/30
27/27 - 7s - loss: 0.6920 - accuracy: 0.5162 - 7s/epoch - 247ms/step
Epoch 2/30
27/27 - 4s - loss: 0.6842 - accuracy: 0.5660 - 4s/epoch - 163ms/step
Epoch 3/30
27/27 - 1s - loss: 0.6255 - accuracy: 0.6470 - 1s/epoch - 46ms/step
Epoch 4/30
27/27 - 2s - loss: 0.5109 - accuracy: 0.7188 - 2s/epoch - 77ms/step
Epoch 5/30
27/27 - 2s - loss: 0.4277 - accuracy: 0.7604 - 2s/epoch - 63ms/step
Epoch 6/30
27/27 - 1s - loss: 0.3178 - accuracy: 0.8438 - 1s/epoch - 53ms/step
Epoch 7/30
27/27 - 1s - loss: 0.2322 - accuracy: 0.8958 - 576ms/epoch - 21ms/step
Epoch 8/30
27/27 - 0s - loss: 0.1779 - accuracy: 0.9363 - 346ms/epoch - 13ms/step
Epoch 9/30
27/27 - 1s - loss: 0.1350 - accuracy: 0.9664 - 796ms/epoch - 29ms/step
Epoch 10/30
27/27 - 0s - loss: 0.1105 - accuracy: 0.9711 - 120ms/epoch - 4ms/step
Epoch 11/30
27/27 - 0s - loss: 0.1395 - accuracy: 0.9792 - 407ms/epoch - 15ms/step
Epoch 12/30
27/27 - 1s - loss: 0.1108 - accuracy: 0.9734 - 1s/epoch - 45ms/step
Epoch 13/30
27/27 - 1s - lo

<ipython-input-18-0da4cc95c167>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 5s - loss: 0.6932 - accuracy: 0.5127 - 5s/epoch - 168ms/step
Epoch 2/30
27/27 - 3s - loss: 0.6710 - accuracy: 0.5706 - 3s/epoch - 95ms/step
Epoch 3/30
27/27 - 2s - loss: 0.6075 - accuracy: 0.6644 - 2s/epoch - 58ms/step
Epoch 4/30
27/27 - 2s - loss: 0.4949 - accuracy: 0.7384 - 2s/epoch - 66ms/step
Epoch 5/30
27/27 - 2s - loss: 0.3359 - accuracy: 0.8576 - 2s/epoch - 76ms/step
Epoch 6/30
27/27 - 1s - loss: 0.2177 - accuracy: 0.8947 - 1s/epoch - 45ms/step
Epoch 7/30
27/27 - 0s - loss: 0.1545 - accuracy: 0.9213 - 343ms/epoch - 13ms/step
Epoch 8/30
27/27 - 0s - loss: 0.1280 - accuracy: 0.9363 - 274ms/epoch - 10ms/step
Epoch 9/30
27/27 - 0s - loss: 0.1195 - accuracy: 0.9236 - 444ms/epoch - 16ms/step
Epoch 10/30
27/27 - 1s - loss: 0.1004 - accuracy: 0.9456 - 1s/epoch - 39ms/step
Epoch 11/30
27/27 - 0s - loss: 0.0879 - accuracy: 0.9595 - 438ms/epoch - 16ms/step
Epoch 12/30
27/27 - 0s - loss: 0.0809 - accuracy: 0.9595 - 296ms/epoch - 11ms/step
Epoch 13/30
27/27 - 0s - loss: 0.0609 - accu

...

27/27 - 8s - loss: 0.6563 - accuracy: 0.6424 - 8s/epoch - 297ms/step
Epoch 2/30
27/27 - 3s - loss: 0.6265 - accuracy: 0.6551 - 3s/epoch - 102ms/step
Epoch 3/30
27/27 - 3s - loss: 0.5684 - accuracy: 0.6551 - 3s/epoch - 95ms/step
Epoch 4/30
27/27 - 2s - loss: 0.4851 - accuracy: 0.6551 - 2s/epoch - 62ms/step
Epoch 5/30
27/27 - 1s - loss: 0.4064 - accuracy: 0.7674 - 796ms/epoch - 29ms/step
Epoch 6/30
27/27 - 0s - loss: 0.3315 - accuracy: 0.9028 - 118ms/epoch - 4ms/step
Epoch 7/30
27/27 - 0s - loss: 0.2885 - accuracy: 0.9618 - 348ms/epoch - 13ms/step
Epoch 8/30
27/27 - 2s - loss: 0.2555 - accuracy: 0.9653 - 2s/epoch - 57ms/step
Epoch 9/30
27/27 - 1s - loss: 0.1720 - accuracy: 0.9583 - 508ms/epoch - 19ms/step
Epoch 10/30
27/27 - 0s - loss: 0.1160 - accuracy: 0.9734 - 152ms/epoch - 6ms/step
Epoch 11/30
27/27 - 0s - loss: 0.1010 - accuracy: 0.9792 - 150ms/epoch - 6ms/step
Epoch 12/30
27/27 - 1s - loss: 0.0947 - accuracy: 0.9873 - 859ms/epoch - 32ms/step
Epoch 13/30
27/27 - 1s - loss: 0.0833 - 

<ipython-input-18-0da4cc95c167>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 7s - loss: 0.6605 - accuracy: 0.6481 - 7s/epoch - 268ms/step
Epoch 2/30
27/27 - 4s - loss: 0.6285 - accuracy: 0.6620 - 4s/epoch - 135ms/step
Epoch 3/30
27/27 - 2s - loss: 0.5771 - accuracy: 0.6620 - 2s/epoch - 72ms/step
Epoch 4/30
27/27 - 2s - loss: 0.5212 - accuracy: 0.6620 - 2s/epoch - 62ms/step
Epoch 5/30
27/27 - 1s - loss: 0.4619 - accuracy: 0.6620 - 1s/epoch - 55ms/step
Epoch 6/30
27/27 - 1s - loss: 0.4157 - accuracy: 0.7616 - 1s/epoch - 46ms/step
Epoch 7/30
27/27 - 1s - loss: 0.3222 - accuracy: 0.8588 - 584ms/epoch - 22ms/step
Epoch 8/30
27/27 - 0s - loss: 0.2407 - accuracy: 0.8657 - 121ms/epoch - 4ms/step
Epoch 9/30
27/27 - 1s - loss: 0.1867 - accuracy: 0.8981 - 584ms/epoch - 22ms/step
Epoch 10/30
27/27 - 1s - loss: 0.1726 - accuracy: 0.9039 - 733ms/epoch - 27ms/step
Epoch 11/30
27/27 - 1s - loss: 0.1534 - accuracy: 0.9062 - 533ms/epoch - 20ms/step
Epoch 12/30
27/27 - 1s - loss: 0.1481 - accuracy: 0.8924 - 880ms/epoch - 33ms/step
Epoch 13/30
27/27 - 1s - loss: 0.1272 - a

<ipython-input-18-0da4cc95c167>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 6s - loss: 0.4689 - accuracy: 0.8438 - 6s/epoch - 230ms/step
Epoch 2/30
27/27 - 3s - loss: 0.4187 - accuracy: 0.8588 - 3s/epoch - 103ms/step
Epoch 3/30
27/27 - 3s - loss: 0.3418 - accuracy: 0.8588 - 3s/epoch - 119ms/step
Epoch 4/30
27/27 - 2s - loss: 0.2885 - accuracy: 0.8588 - 2s/epoch - 63ms/step
Epoch 5/30
27/27 - 1s - loss: 0.1957 - accuracy: 0.8808 - 790ms/epoch - 29ms/step
Epoch 6/30
27/27 - 1s - loss: 0.1260 - accuracy: 0.9375 - 1s/epoch - 38ms/step
Epoch 7/30
27/27 - 1s - loss: 0.1054 - accuracy: 0.9375 - 1s/epoch - 38ms/step
Epoch 8/30
27/27 - 0s - loss: 0.0960 - accuracy: 0.9491 - 357ms/epoch - 13ms/step
Epoch 9/30
27/27 - 1s - loss: 0.0670 - accuracy: 0.9688 - 1s/epoch - 38ms/step
Epoch 10/30
27/27 - 1s - loss: 0.0541 - accuracy: 0.9780 - 796ms/epoch - 29ms/step
Epoch 11/30
27/27 - 0s - loss: 0.0534 - accuracy: 0.9792 - 122ms/epoch - 5ms/step
Epoch 12/30
27/27 - 0s - loss: 0.0557 - accuracy: 0.9734 - 345ms/epoch - 13ms/step
Epoch 13/30
27/27 - 1s - loss: 0.0469 - acc

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.566327,0.550747,0.584184,0.555240
1,original2,0.549745,0.548628,0.543367,0.543293
2,original3,0.593112,0.555160,0.586735,0.532973
3,original4,0.572704,0.562971,0.543367,0.506606
4,original5,0.454082,0.432172,0.436224,0.396137


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.547194,0.529936,0.538776,0.506850
std,0.054318,0.054927,0.061075,0.064439
min,0.454082,0.432172,0.436224,0.396137
25%,0.549745,0.548628,0.543367,0.506606
50%,0.566327,0.550747,0.543367,0.532973
75%,0.572704,0.555160,0.584184,0.543293
max,0.593112,0.562971,0.586735,0.555240


# Contrastive LAMBADA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/lambada/base_ironic_'+str(perc)+'_lambada_contrastive'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/lambada/base_ironic_'+str(perc)+'_lambada_contrastive'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/lambada/base_ironic_'+str(perc)+'_lambada_contrastive'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/lambada/base_ironic_'+str(perc)+'_lambada_contrastive'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/lambada/base_ironic_'+str(perc)+'_lambada_contrastive'+str(i+5)+'.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 7s - loss: 0.6947 - accuracy: 0.4745 - 7s/epoch - 245ms/step
Epoch 2/30
27/27 - 3s - loss: 0.6931 - accuracy: 0.5104 - 3s/epoch - 113ms/step
Epoch 3/30
27/27 - 4s - loss: 0.6931 - accuracy: 0.5104 - 4s/epoch - 137ms/step
Epoch 4/30
27/27 - 1s - loss: 0.6930 - accuracy: 0.5104 - 977ms/epoch - 36ms/step
Epoch 5/30
27/27 - 1s - loss: 0.6930 - accuracy: 0.5104 - 1s/epoch - 45ms/step
Epoch 6/30
27/27 - 1s - loss: 0.6930 - accuracy: 0.5104 - 988ms/epoch - 37ms/step
Epoch 7/30
27/27 - 0s - loss: 0.6930 - accuracy: 0.5104 - 345ms/epoch - 13ms/step
Epoch 8/30
27/27 - 0s - loss: 0.6930 - accuracy: 0.5104 - 329ms/epoch - 12ms/step
Epoch 9/30
27/27 - 0s - loss: 0.6930 - accuracy: 0.5104 - 332ms/epoch - 12ms/step
Epoch 10/30
27/27 - 1s - loss: 0.6930 - accuracy: 0.5104 - 556ms/epoch - 21ms/step
Epoch 11/30
27/27 - 0s - loss: 0.6930 - accuracy: 0.5104 - 118ms/epoch - 4ms/step
Epoch 12/30
27/27 - 1s - loss: 0.6930 - accuracy: 0.5104 - 548ms/epoch - 20ms/step
Epoch 13/30
27/27 - 0s - loss: 0.6

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 6s - loss: 0.6903 - accuracy: 0.5185 - 6s/epoch - 232ms/step
Epoch 2/30
27/27 - 4s - loss: 0.6679 - accuracy: 0.5440 - 4s/epoch - 138ms/step
Epoch 3/30
27/27 - 4s - loss: 0.5539 - accuracy: 0.7118 - 4s/epoch - 147ms/step
Epoch 4/30
27/27 - 1s - loss: 0.4221 - accuracy: 0.8113 - 568ms/epoch - 21ms/step
Epoch 5/30
27/27 - 1s - loss: 0.3103 - accuracy: 0.8808 - 1s/epoch - 52ms/step
Epoch 6/30
27/27 - 1s - loss: 0.2272 - accuracy: 0.9410 - 996ms/epoch - 37ms/step
Epoch 7/30
27/27 - 0s - loss: 0.1785 - accuracy: 0.9549 - 119ms/epoch - 4ms/step
Epoch 8/30
27/27 - 1s - loss: 0.1240 - accuracy: 0.9734 - 988ms/epoch - 37ms/step
Epoch 9/30
27/27 - 1s - loss: 0.1270 - accuracy: 0.9734 - 564ms/epoch - 21ms/step
Epoch 10/30
27/27 - 1s - loss: 0.1145 - accuracy: 0.9769 - 1s/epoch - 37ms/step
Epoch 11/30
27/27 - 1s - loss: 0.0930 - accuracy: 0.9838 - 760ms/epoch - 28ms/step
Epoch 12/30
27/27 - 0s - loss: 0.0849 - accuracy: 0.9838 - 124ms/epoch - 5ms/step
Epoch 13/30
27/27 - 0s - loss: 0.0907 

...

27/27 - 6s - loss: 0.6646 - accuracy: 0.6343 - 6s/epoch - 222ms/step
Epoch 2/30
27/27 - 5s - loss: 0.6610 - accuracy: 0.6435 - 5s/epoch - 179ms/step
Epoch 3/30
27/27 - 1s - loss: 0.6645 - accuracy: 0.6435 - 1s/epoch - 52ms/step
Epoch 4/30
27/27 - 1s - loss: 0.6372 - accuracy: 0.6435 - 1s/epoch - 52ms/step
Epoch 5/30
27/27 - 1s - loss: 0.5642 - accuracy: 0.6435 - 979ms/epoch - 36ms/step
Epoch 6/30
27/27 - 1s - loss: 0.4888 - accuracy: 0.6435 - 552ms/epoch - 20ms/step
Epoch 7/30
27/27 - 0s - loss: 0.3458 - accuracy: 0.8542 - 125ms/epoch - 5ms/step
Epoch 8/30
27/27 - 1s - loss: 0.2113 - accuracy: 0.9028 - 1s/epoch - 45ms/step
Epoch 9/30
27/27 - 1s - loss: 0.1701 - accuracy: 0.9271 - 540ms/epoch - 20ms/step
Epoch 10/30
27/27 - 0s - loss: 0.1233 - accuracy: 0.9294 - 355ms/epoch - 13ms/step
Epoch 11/30
27/27 - 0s - loss: 0.1133 - accuracy: 0.9433 - 333ms/epoch - 12ms/step
Epoch 12/30
27/27 - 1s - loss: 0.0993 - accuracy: 0.9468 - 769ms/epoch - 28ms/step
Epoch 13/30
27/27 - 0s - loss: 0.0936 

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 6s - loss: 0.6931 - accuracy: 0.5220 - 6s/epoch - 231ms/step
Epoch 2/30
27/27 - 5s - loss: 0.6858 - accuracy: 0.5451 - 5s/epoch - 191ms/step
Epoch 3/30
27/27 - 2s - loss: 0.6736 - accuracy: 0.5660 - 2s/epoch - 76ms/step
Epoch 4/30
27/27 - 1s - loss: 0.6338 - accuracy: 0.6238 - 981ms/epoch - 36ms/step
Epoch 5/30
27/27 - 1s - loss: 0.5601 - accuracy: 0.6481 - 1s/epoch - 46ms/step
Epoch 6/30
27/27 - 1s - loss: 0.4455 - accuracy: 0.7083 - 784ms/epoch - 29ms/step
Epoch 7/30
27/27 - 1s - loss: 0.3861 - accuracy: 0.7697 - 778ms/epoch - 29ms/step
Epoch 8/30
27/27 - 1s - loss: 0.3229 - accuracy: 0.8507 - 553ms/epoch - 20ms/step
Epoch 9/30
27/27 - 1s - loss: 0.2735 - accuracy: 0.8970 - 560ms/epoch - 21ms/step
Epoch 10/30
27/27 - 0s - loss: 0.2715 - accuracy: 0.9028 - 338ms/epoch - 13ms/step
Epoch 11/30
27/27 - 1s - loss: 0.1996 - accuracy: 0.9213 - 553ms/epoch - 20ms/step
Epoch 12/30
27/27 - 0s - loss: 0.1475 - accuracy: 0.9225 - 123ms/epoch - 5ms/step
Epoch 13/30
27/27 - 1s - loss: 0.17

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 6s - loss: 0.6909 - accuracy: 0.5544 - 6s/epoch - 211ms/step
Epoch 2/30
27/27 - 4s - loss: 0.6623 - accuracy: 0.5671 - 4s/epoch - 135ms/step
Epoch 3/30
27/27 - 1s - loss: 0.6277 - accuracy: 0.5671 - 1s/epoch - 52ms/step
Epoch 4/30
27/27 - 1s - loss: 0.5834 - accuracy: 0.5683 - 981ms/epoch - 36ms/step
Epoch 5/30
27/27 - 2s - loss: 0.5625 - accuracy: 0.7095 - 2s/epoch - 73ms/step
Epoch 6/30
27/27 - 1s - loss: 0.4924 - accuracy: 0.7593 - 562ms/epoch - 21ms/step
Epoch 7/30
27/27 - 1s - loss: 0.4407 - accuracy: 0.8206 - 786ms/epoch - 29ms/step
Epoch 8/30
27/27 - 1s - loss: 0.4035 - accuracy: 0.8333 - 1s/epoch - 45ms/step
Epoch 9/30
27/27 - 1s - loss: 0.3482 - accuracy: 0.8148 - 772ms/epoch - 29ms/step
Epoch 10/30
27/27 - 1s - loss: 0.2918 - accuracy: 0.8322 - 555ms/epoch - 21ms/step
Epoch 11/30
27/27 - 0s - loss: 0.2210 - accuracy: 0.8484 - 336ms/epoch - 12ms/step
Epoch 12/30
27/27 - 1s - loss: 0.1813 - accuracy: 0.8634 - 559ms/epoch - 21ms/step
Epoch 13/30
27/27 - 0s - loss: 0.1544

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.603316,0.376293,0.515306,0.509122
1,original2,0.551020,0.537510,0.577806,0.571781
2,original3,0.485969,0.484418,0.468112,0.466925
3,original4,0.511480,0.509404,0.547194,0.537932
4,original5,0.552296,0.550816,0.572704,0.570433


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.540816,0.491688,0.536224,0.531238
std,0.044752,0.069424,0.045434,0.044300
min,0.485969,0.376293,0.468112,0.466925
25%,0.511480,0.484418,0.515306,0.509122
50%,0.551020,0.509404,0.547194,0.537932
75%,0.552296,0.537510,0.572704,0.570433
max,0.603316,0.550816,0.577806,0.571781


# LLAMA 2 Paraphraser

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/gptparaphrase/base_ironic_'+str(perc)+'_gptparaphrase'+str(i+1)+'.pkl',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/gptparaphrase/base_ironic_'+str(perc)+'_gptparaphrase'+str(i+2)+'.pkl',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/gptparaphrase/base_ironic_'+str(perc)+'_gptparaphrase'+str(i+3)+'.pkl',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/gptparaphrase/base_ironic_'+str(perc)+'_gptparaphrase'+str(i+4)+'.pkl',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/gptparaphrase/base_ironic_'+str(perc)+'_gptparaphrase'+str(i+5)+'.pkl',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal_pkl(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-9-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
27/27 - 8s - loss: 0.6927 - accuracy: 0.5024 - 8s/epoch - 292ms/step
Epoch 2/30
27/27 - 3s - loss: 0.6821 - accuracy: 0.5488 - 3s/epoch - 123ms/step
Epoch 3/30
27/27 - 5s - loss: 0.6315 - accuracy: 0.6405 - 5s/epoch - 191ms/step
Epoch 4/30
27/27 - 1s - loss: 0.4656 - accuracy: 0.7155 - 1s/epoch - 49ms/step
Epoch 5/30
27/27 - 1s - loss: 0.3124 - accuracy: 0.8786 - 1s/epoch - 45ms/step
Epoch 6/30
27/27 - 2s - loss: 0.2482 - accuracy: 0.9143 - 2s/epoch - 60ms/step
Epoch 7/30
27/27 - 1s - loss: 0.2208 - accuracy: 0.9238 - 798ms/epoch - 30ms/step
Epoch 8/30
27/27 - 1s - loss: 0.2297 - accuracy: 0.9274 - 1s/epoch - 37ms/step
Epoch 9/30
27/27 - 1s - loss: 0.1948 - accuracy: 0.9429 - 574ms/epoch - 21ms/step
Epoch 10/30
27/27 - 1s - loss: 0.1901 - accuracy: 0.9464 - 1s/epoch - 37ms/step
Epoch 11/30
27/27 - 1s - loss: 0.2022 - accuracy: 0.9321 - 573ms/epoch - 21ms/step
Epoch 12/30
27/27 - 1s - loss: 0.2026 - accuracy: 0.9321 - 1s/epoch - 46ms/step
Epoch 13/30
27/27 - 0s - loss: 0.1946

<ipython-input-9-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 7s - loss: 0.7049 - accuracy: 0.4846 - 7s/epoch - 248ms/step
Epoch 2/30
27/27 - 3s - loss: 0.6537 - accuracy: 0.6303 - 3s/epoch - 95ms/step
Epoch 3/30
27/27 - 2s - loss: 0.4858 - accuracy: 0.8069 - 2s/epoch - 77ms/step
Epoch 4/30
27/27 - 2s - loss: 0.2166 - accuracy: 0.9408 - 2s/epoch - 64ms/step
Epoch 5/30
27/27 - 1s - loss: 0.1144 - accuracy: 0.9763 - 782ms/epoch - 29ms/step
Epoch 6/30
27/27 - 0s - loss: 0.1022 - accuracy: 0.9822 - 498ms/epoch - 18ms/step
Epoch 7/30
27/27 - 2s - loss: 0.0809 - accuracy: 0.9905 - 2s/epoch - 69ms/step
Epoch 8/30
27/27 - 1s - loss: 0.0999 - accuracy: 0.9870 - 1s/epoch - 47ms/step
Epoch 9/30
27/27 - 0s - loss: 0.0840 - accuracy: 0.9929 - 174ms/epoch - 6ms/step
Epoch 10/30
27/27 - 0s - loss: 0.0738 - accuracy: 0.9976 - 440ms/epoch - 16ms/step
Epoch 11/30
27/27 - 0s - loss: 0.0770 - accuracy: 0.9905 - 384ms/epoch - 14ms/step
Epoch 12/30
27/27 - 0s - loss: 0.0846 - accuracy: 0.9858 - 342ms/epoch - 13ms/step
Epoch 13/30
27/27 - 0s - loss: 0.0784 - ac

...

Epoch 1/30
27/27 - 7s - loss: 0.6949 - accuracy: 0.5095 - 7s/epoch - 260ms/step
Epoch 2/30
27/27 - 5s - loss: 0.6905 - accuracy: 0.5227 - 5s/epoch - 175ms/step
Epoch 3/30
27/27 - 3s - loss: 0.6722 - accuracy: 0.5453 - 3s/epoch - 117ms/step
Epoch 4/30
27/27 - 1s - loss: 0.5757 - accuracy: 0.6516 - 1s/epoch - 44ms/step
Epoch 5/30
27/27 - 2s - loss: 0.3505 - accuracy: 0.8019 - 2s/epoch - 84ms/step
Epoch 6/30
27/27 - 1s - loss: 0.1880 - accuracy: 0.9129 - 1s/epoch - 52ms/step
Epoch 7/30
27/27 - 1s - loss: 0.1002 - accuracy: 0.9463 - 1s/epoch - 52ms/step
Epoch 8/30
27/27 - 1s - loss: 0.0665 - accuracy: 0.9666 - 553ms/epoch - 20ms/step
Epoch 9/30
27/27 - 1s - loss: 0.0468 - accuracy: 0.9726 - 557ms/epoch - 21ms/step
Epoch 10/30
27/27 - 1s - loss: 0.0356 - accuracy: 0.9737 - 776ms/epoch - 29ms/step
Epoch 11/30
27/27 - 1s - loss: 0.0393 - accuracy: 0.9666 - 781ms/epoch - 29ms/step
Epoch 12/30
27/27 - 1s - loss: 0.0360 - accuracy: 0.9642 - 1s/epoch - 43ms/step
Epoch 13/30
27/27 - 1s - loss: 0.0

<ipython-input-9-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 7s - loss: 0.6958 - accuracy: 0.5521 - 7s/epoch - 245ms/step
Epoch 2/30
27/27 - 4s - loss: 0.6425 - accuracy: 0.6359 - 4s/epoch - 132ms/step
Epoch 3/30
27/27 - 4s - loss: 0.5430 - accuracy: 0.6731 - 4s/epoch - 146ms/step
Epoch 4/30
27/27 - 2s - loss: 0.3195 - accuracy: 0.8826 - 2s/epoch - 63ms/step
Epoch 5/30
27/27 - 2s - loss: 0.1401 - accuracy: 0.9557 - 2s/epoch - 84ms/step
Epoch 6/30
27/27 - 1s - loss: 0.1067 - accuracy: 0.9856 - 994ms/epoch - 37ms/step
Epoch 7/30
27/27 - 2s - loss: 0.0945 - accuracy: 0.9832 - 2s/epoch - 69ms/step
Epoch 8/30
27/27 - 1s - loss: 0.0843 - accuracy: 0.9880 - 998ms/epoch - 37ms/step
Epoch 9/30
27/27 - 1s - loss: 0.0791 - accuracy: 0.9952 - 991ms/epoch - 37ms/step
Epoch 10/30
27/27 - 1s - loss: 0.0856 - accuracy: 0.9928 - 1s/epoch - 37ms/step
Epoch 11/30
27/27 - 1s - loss: 0.0837 - accuracy: 0.9964 - 894ms/epoch - 33ms/step
Epoch 12/30
27/27 - 1s - loss: 0.0680 - accuracy: 0.9952 - 1s/epoch - 46ms/step
Epoch 13/30
27/27 - 1s - loss: 0.0723 - accur

<ipython-input-9-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 8s - loss: 0.6960 - accuracy: 0.4918 - 8s/epoch - 308ms/step
Epoch 2/30
27/27 - 3s - loss: 0.6919 - accuracy: 0.5223 - 3s/epoch - 110ms/step
Epoch 3/30
27/27 - 3s - loss: 0.6872 - accuracy: 0.5716 - 3s/epoch - 98ms/step
Epoch 4/30
27/27 - 3s - loss: 0.6666 - accuracy: 0.6150 - 3s/epoch - 126ms/step
Epoch 5/30
27/27 - 1s - loss: 0.4903 - accuracy: 0.7512 - 1s/epoch - 53ms/step
Epoch 6/30
27/27 - 2s - loss: 0.2836 - accuracy: 0.8580 - 2s/epoch - 60ms/step
Epoch 7/30
27/27 - 1s - loss: 0.1823 - accuracy: 0.8991 - 788ms/epoch - 29ms/step
Epoch 8/30
27/27 - 1s - loss: 0.1661 - accuracy: 0.9143 - 1s/epoch - 52ms/step
Epoch 9/30
27/27 - 0s - loss: 0.1215 - accuracy: 0.9390 - 358ms/epoch - 13ms/step
Epoch 10/30
27/27 - 0s - loss: 0.1469 - accuracy: 0.9284 - 338ms/epoch - 13ms/step
Epoch 11/30
27/27 - 1s - loss: 0.1386 - accuracy: 0.9401 - 775ms/epoch - 29ms/step
Epoch 12/30
27/27 - 0s - loss: 0.0987 - accuracy: 0.9695 - 342ms/epoch - 13ms/step
Epoch 13/30
27/27 - 0s - loss: 0.0881 - ac

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.543367,0.536416,0.572704,0.560043
1,original2,0.540816,0.539735,0.582908,0.582853
2,original3,0.452806,0.426141,0.531888,0.529400
3,original4,0.516582,0.494741,0.545918,0.512942
4,original5,0.553571,0.550762,0.607143,0.587278


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.521429,0.509559,0.568112,0.554503
std,0.040691,0.051254,0.029867,0.032658
min,0.452806,0.426141,0.531888,0.512942
25%,0.516582,0.494741,0.545918,0.529400
50%,0.540816,0.536416,0.572704,0.560043
75%,0.543367,0.539735,0.582908,0.582853
max,0.553571,0.550762,0.607143,0.587278


# ProtAugment

In [18]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/protaugment/base_ironic_'+str(perc)+'_protaugment'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/protaugment/base_ironic_'+str(perc)+'_protaugment'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/protaugment/base_ironic_'+str(perc)+'_protaugment'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/protaugment/base_ironic_'+str(perc)+'_protaugment'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/protaugment/base_ironic_'+str(perc)+'_protaugment'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet', 4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


26/26 - 7s - loss: 0.6896 - accuracy: 0.5122 - 7s/epoch - 259ms/step
Epoch 2/30
26/26 - 4s - loss: 0.6395 - accuracy: 0.6302 - 4s/epoch - 148ms/step
Epoch 3/30
26/26 - 3s - loss: 0.5315 - accuracy: 0.8589 - 3s/epoch - 97ms/step
Epoch 4/30
26/26 - 2s - loss: 0.4050 - accuracy: 0.9404 - 2s/epoch - 76ms/step
Epoch 5/30
26/26 - 1s - loss: 0.2857 - accuracy: 0.9659 - 876ms/epoch - 34ms/step
Epoch 6/30
26/26 - 1s - loss: 0.0898 - accuracy: 0.9866 - 679ms/epoch - 26ms/step
Epoch 7/30
26/26 - 1s - loss: 0.0648 - accuracy: 0.9830 - 1s/epoch - 46ms/step
Epoch 8/30
26/26 - 1s - loss: 0.0641 - accuracy: 0.9805 - 674ms/epoch - 26ms/step
Epoch 9/30
26/26 - 1s - loss: 0.0447 - accuracy: 0.9927 - 508ms/epoch - 20ms/step
Epoch 10/30
26/26 - 2s - loss: 0.0530 - accuracy: 0.9866 - 2s/epoch - 58ms/step
Epoch 11/30
26/26 - 0s - loss: 0.0547 - accuracy: 0.9793 - 471ms/epoch - 18ms/step
Epoch 12/30
26/26 - 1s - loss: 0.0492 - accuracy: 0.9891 - 1s/epoch - 54ms/step
Epoch 13/30
26/26 - 1s - loss: 0.0414 - acc

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


26/26 - 8s - loss: 0.6896 - accuracy: 0.5250 - 8s/epoch - 295ms/step
Epoch 2/30
26/26 - 4s - loss: 0.6597 - accuracy: 0.5873 - 4s/epoch - 139ms/step
Epoch 3/30
26/26 - 2s - loss: 0.5303 - accuracy: 0.7399 - 2s/epoch - 81ms/step
Epoch 4/30
26/26 - 2s - loss: 0.2868 - accuracy: 0.8486 - 2s/epoch - 84ms/step
Epoch 5/30
26/26 - 4s - loss: 0.1641 - accuracy: 0.9133 - 4s/epoch - 142ms/step
Epoch 6/30
26/26 - 1s - loss: 0.1097 - accuracy: 0.9731 - 1s/epoch - 40ms/step
Epoch 7/30
26/26 - 1s - loss: 0.0914 - accuracy: 0.9853 - 680ms/epoch - 26ms/step
Epoch 8/30
26/26 - 0s - loss: 0.0905 - accuracy: 0.9853 - 315ms/epoch - 12ms/step
Epoch 9/30
26/26 - 1s - loss: 0.0899 - accuracy: 0.9927 - 881ms/epoch - 34ms/step
Epoch 10/30
26/26 - 0s - loss: 0.0814 - accuracy: 0.9902 - 304ms/epoch - 12ms/step
Epoch 11/30
26/26 - 0s - loss: 0.0816 - accuracy: 0.9915 - 309ms/epoch - 12ms/step
Epoch 12/30
26/26 - 1s - loss: 0.0740 - accuracy: 0.9915 - 1s/epoch - 40ms/step
Epoch 13/30
26/26 - 1s - loss: 0.0765 - ac

...

26/26 - 6s - loss: 0.6958 - accuracy: 0.4725 - 6s/epoch - 221ms/step
Epoch 2/30
26/26 - 3s - loss: 0.6665 - accuracy: 0.5470 - 3s/epoch - 115ms/step
Epoch 3/30
26/26 - 3s - loss: 0.5547 - accuracy: 0.6801 - 3s/epoch - 134ms/step
Epoch 4/30
26/26 - 2s - loss: 0.3696 - accuracy: 0.7424 - 2s/epoch - 77ms/step
Epoch 5/30
26/26 - 2s - loss: 0.2638 - accuracy: 0.7729 - 2s/epoch - 61ms/step
Epoch 6/30
26/26 - 1s - loss: 0.2254 - accuracy: 0.9158 - 1s/epoch - 54ms/step
Epoch 7/30
26/26 - 1s - loss: 0.2158 - accuracy: 0.9426 - 856ms/epoch - 33ms/step
Epoch 8/30
26/26 - 1s - loss: 0.1889 - accuracy: 0.9731 - 688ms/epoch - 26ms/step
Epoch 9/30
26/26 - 0s - loss: 0.1765 - accuracy: 0.9585 - 318ms/epoch - 12ms/step
Epoch 10/30
26/26 - 1s - loss: 0.1641 - accuracy: 0.9719 - 511ms/epoch - 20ms/step
Epoch 11/30
26/26 - 1s - loss: 0.1885 - accuracy: 0.9499 - 689ms/epoch - 27ms/step
Epoch 12/30
26/26 - 0s - loss: 0.1682 - accuracy: 0.9609 - 126ms/epoch - 5ms/step
Epoch 13/30
26/26 - 1s - loss: 0.1496 - 

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


26/26 - 7s - loss: 0.6709 - accuracy: 0.5898 - 7s/epoch - 272ms/step
Epoch 2/30
26/26 - 3s - loss: 0.5474 - accuracy: 0.7770 - 3s/epoch - 134ms/step
Epoch 3/30
26/26 - 2s - loss: 0.3657 - accuracy: 0.9108 - 2s/epoch - 83ms/step
Epoch 4/30
26/26 - 2s - loss: 0.2579 - accuracy: 0.9616 - 2s/epoch - 62ms/step
Epoch 5/30
26/26 - 1s - loss: 0.1790 - accuracy: 0.9876 - 1s/epoch - 55ms/step
Epoch 6/30
26/26 - 2s - loss: 0.1720 - accuracy: 0.9876 - 2s/epoch - 96ms/step
Epoch 7/30
26/26 - 2s - loss: 0.1474 - accuracy: 0.9950 - 2s/epoch - 64ms/step
Epoch 8/30
26/26 - 1s - loss: 0.1504 - accuracy: 0.9913 - 1s/epoch - 42ms/step
Epoch 9/30
26/26 - 1s - loss: 0.1436 - accuracy: 0.9926 - 677ms/epoch - 26ms/step
Epoch 10/30
26/26 - 0s - loss: 0.1450 - accuracy: 0.9901 - 316ms/epoch - 12ms/step
Epoch 11/30
26/26 - 1s - loss: 0.1263 - accuracy: 0.9876 - 692ms/epoch - 27ms/step
Epoch 12/30
26/26 - 1s - loss: 0.1287 - accuracy: 0.9926 - 1s/epoch - 54ms/step
Epoch 13/30
26/26 - 1s - loss: 0.1357 - accuracy:

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


26/26 - 6s - loss: 0.6787 - accuracy: 0.5564 - 6s/epoch - 217ms/step
Epoch 2/30
26/26 - 3s - loss: 0.5131 - accuracy: 0.8113 - 3s/epoch - 119ms/step
Epoch 3/30
26/26 - 5s - loss: 0.2040 - accuracy: 0.9559 - 5s/epoch - 179ms/step
Epoch 4/30
26/26 - 1s - loss: 0.1002 - accuracy: 0.9828 - 1s/epoch - 48ms/step
Epoch 5/30
26/26 - 2s - loss: 0.0660 - accuracy: 0.9939 - 2s/epoch - 62ms/step
Epoch 6/30
26/26 - 1s - loss: 0.0593 - accuracy: 0.9939 - 1s/epoch - 42ms/step
Epoch 7/30
26/26 - 1s - loss: 0.0520 - accuracy: 0.9963 - 1s/epoch - 49ms/step
Epoch 8/30
26/26 - 1s - loss: 0.0465 - accuracy: 0.9963 - 935ms/epoch - 36ms/step
Epoch 9/30
26/26 - 0s - loss: 0.0376 - accuracy: 0.9926 - 496ms/epoch - 19ms/step
Epoch 10/30
26/26 - 1s - loss: 0.0454 - accuracy: 0.9914 - 1s/epoch - 40ms/step
Epoch 11/30
26/26 - 0s - loss: 0.0378 - accuracy: 0.9951 - 498ms/epoch - 19ms/step
Epoch 12/30
26/26 - 0s - loss: 0.0448 - accuracy: 0.9975 - 491ms/epoch - 19ms/step
Epoch 13/30
26/26 - 1s - loss: 0.0463 - accur

In [19]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [20]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.519133,0.493073,0.517857,0.501950
std,0.052583,0.039159,0.033310,0.027825
min,0.456633,0.456406,0.489796,0.468763
25%,0.492347,0.469705,0.493622,0.482488
50%,0.503827,0.479614,0.511480,0.500737
75%,0.552296,0.503729,0.521684,0.519808
max,0.590561,0.555909,0.572704,0.537956


# GPTNEOKEY_PKA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/teste_algo/base_ironic_'+str(perc)+'_gptneokey'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/teste_algo/base_ironic_'+str(perc)+'_gptneokey'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/teste_algo/base_ironic_'+str(perc)+'_gptneokey'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/teste_algo/base_ironic_'+str(perc)+'_gptneokey'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/teste_algo/base_ironic_'+str(perc)+'_gptneokey'+str(i+5)+'.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/ironic/base_ironic_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-9-7cb4088b5ab9>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
27/27 - 3s - loss: 0.6948 - accuracy: 0.5081 - 3s/epoch - 126ms/step
Epoch 2/30
27/27 - 2s - loss: 0.6931 - accuracy: 0.4954 - 2s/epoch - 74ms/step
Epoch 3/30
27/27 - 2s - loss: 0.6921 - accuracy: 0.5035 - 2s/epoch - 75ms/step
Epoch 4/30
27/27 - 2s - loss: 0.6844 - accuracy: 0.5370 - 2s/epoch - 74ms/step
Epoch 5/30
27/27 - 3s - loss: 0.5753 - accuracy: 0.6632 - 3s/epoch - 101ms/step
Epoch 6/30
27/27 - 4s - loss: 0.3045 - accuracy: 0.8785 - 4s/epoch - 134ms/step
Epoch 7/30
27/27 - 3s - loss: 0.1746 - accuracy: 0.9572 - 3s/epoch - 129ms/step
Epoch 8/30
27/27 - 2s - loss: 0.1125 - accuracy: 0.9815 - 2s/epoch - 81ms/step
Epoch 9/30
27/27 - 2s - loss: 0.0976 - accuracy: 0.9884 - 2s/epoch - 75ms/step
Epoch 10/30
27/27 - 2s - loss: 0.0998 - accuracy: 0.9873 - 2s/epoch - 75ms/step
Epoch 11/30
27/27 - 2s - loss: 0.0900 - accuracy: 0.9919 - 2s/epoch - 74ms/step
Epoch 12/30
27/27 - 2s - loss: 0.0900 - accuracy: 0.9873 - 2s/epoch - 74ms/step
Epoch 13/30
27/27 - 4s - loss: 0.0833 - accur

<ipython-input-9-7cb4088b5ab9>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 3s - loss: 0.6959 - accuracy: 0.5069 - 3s/epoch - 112ms/step
Epoch 2/30
27/27 - 2s - loss: 0.6801 - accuracy: 0.5486 - 2s/epoch - 73ms/step
Epoch 3/30
27/27 - 3s - loss: 0.5838 - accuracy: 0.7118 - 3s/epoch - 123ms/step
Epoch 4/30
27/27 - 4s - loss: 0.3895 - accuracy: 0.8160 - 4s/epoch - 133ms/step
Epoch 5/30
27/27 - 3s - loss: 0.2552 - accuracy: 0.8576 - 3s/epoch - 114ms/step
Epoch 6/30
27/27 - 2s - loss: 0.1889 - accuracy: 0.8785 - 2s/epoch - 73ms/step
Epoch 7/30
27/27 - 2s - loss: 0.1771 - accuracy: 0.8646 - 2s/epoch - 74ms/step
Epoch 8/30
27/27 - 2s - loss: 0.1525 - accuracy: 0.8889 - 2s/epoch - 75ms/step
Epoch 9/30
27/27 - 2s - loss: 0.1740 - accuracy: 0.8831 - 2s/epoch - 76ms/step
Epoch 10/30
27/27 - 3s - loss: 0.1404 - accuracy: 0.9317 - 3s/epoch - 96ms/step
Epoch 11/30
27/27 - 4s - loss: 0.1322 - accuracy: 0.9398 - 4s/epoch - 133ms/step
Epoch 12/30
27/27 - 4s - loss: 0.1218 - accuracy: 0.9410 - 4s/epoch - 131ms/step
Epoch 13/30
27/27 - 2s - loss: 0.1424 - accuracy: 0.93

...

27/27 - 3s - loss: 0.6952 - accuracy: 0.5116 - 3s/epoch - 117ms/step
Epoch 2/30
27/27 - 2s - loss: 0.6857 - accuracy: 0.5544 - 2s/epoch - 76ms/step
Epoch 3/30
27/27 - 3s - loss: 0.6647 - accuracy: 0.5903 - 3s/epoch - 123ms/step
Epoch 4/30
27/27 - 4s - loss: 0.5623 - accuracy: 0.6898 - 4s/epoch - 136ms/step
Epoch 5/30
27/27 - 3s - loss: 0.4350 - accuracy: 0.7350 - 3s/epoch - 116ms/step
Epoch 6/30
27/27 - 2s - loss: 0.3368 - accuracy: 0.7685 - 2s/epoch - 77ms/step
Epoch 7/30
27/27 - 2s - loss: 0.2558 - accuracy: 0.7431 - 2s/epoch - 75ms/step
Epoch 8/30
27/27 - 2s - loss: 0.1960 - accuracy: 0.9549 - 2s/epoch - 75ms/step
Epoch 9/30
27/27 - 2s - loss: 0.1696 - accuracy: 0.9769 - 2s/epoch - 74ms/step
Epoch 10/30
27/27 - 3s - loss: 0.1778 - accuracy: 0.9919 - 3s/epoch - 96ms/step
Epoch 11/30
27/27 - 4s - loss: 0.1594 - accuracy: 0.9826 - 4s/epoch - 135ms/step
Epoch 12/30
27/27 - 4s - loss: 0.1508 - accuracy: 0.9861 - 4s/epoch - 132ms/step
Epoch 13/30
27/27 - 2s - loss: 0.1482 - accuracy: 0.98

<ipython-input-9-7cb4088b5ab9>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
27/27 - 5s - loss: 0.6964 - accuracy: 0.5058 - 5s/epoch - 187ms/step
Epoch 2/30
27/27 - 2s - loss: 0.6888 - accuracy: 0.5255 - 2s/epoch - 77ms/step
Epoch 3/30
27/27 - 2s - loss: 0.6739 - accuracy: 0.5498 - 2s/epoch - 76ms/step
Epoch 4/30
27/27 - 2s - loss: 0.5565 - accuracy: 0.6725 - 2s/epoch - 74ms/step
Epoch 5/30
27/27 - 2s - loss: 0.2883 - accuracy: 0.8819 - 2s/epoch - 74ms/step
Epoch 6/30
27/27 - 2s - loss: 0.1595 - accuracy: 0.9329 - 2s/epoch - 85ms/step
Epoch 7/30
27/27 - 4s - loss: 0.0955 - accuracy: 0.9491 - 4s/epoch - 139ms/step
Epoch 8/30
27/27 - 4s - loss: 0.0770 - accuracy: 0.9537 - 4s/epoch - 133ms/step
Epoch 9/30
27/27 - 3s - loss: 0.0718 - accuracy: 0.9595 - 3s/epoch - 94ms/step
Epoch 10/30
27/27 - 2s - loss: 0.0689 - accuracy: 0.9572 - 2s/epoch - 75ms/step
Epoch 11/30
27/27 - 2s - loss: 0.0624 - accuracy: 0.9641 - 2s/epoch - 76ms/step
Epoch 12/30
27/27 - 2s - loss: 0.0658 - accuracy: 0.9653 - 2s/epoch - 76ms/step
Epoch 13/30
27/27 - 2s - loss: 0.0668 - accura

<ipython-input-9-7cb4088b5ab9>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


27/27 - 3s - loss: 0.6972 - accuracy: 0.4942 - 3s/epoch - 114ms/step
Epoch 2/30
27/27 - 2s - loss: 0.6922 - accuracy: 0.5243 - 2s/epoch - 75ms/step
Epoch 3/30
27/27 - 3s - loss: 0.6871 - accuracy: 0.5417 - 3s/epoch - 107ms/step
Epoch 4/30
27/27 - 4s - loss: 0.6638 - accuracy: 0.5764 - 4s/epoch - 134ms/step
Epoch 5/30
27/27 - 3s - loss: 0.5851 - accuracy: 0.6829 - 3s/epoch - 129ms/step
Epoch 6/30
27/27 - 2s - loss: 0.5416 - accuracy: 0.7280 - 2s/epoch - 74ms/step
Epoch 7/30
27/27 - 2s - loss: 0.3928 - accuracy: 0.7477 - 2s/epoch - 73ms/step
Epoch 8/30
27/27 - 2s - loss: 0.2810 - accuracy: 0.7222 - 2s/epoch - 74ms/step
Epoch 9/30
27/27 - 2s - loss: 0.1964 - accuracy: 0.7685 - 2s/epoch - 75ms/step
Epoch 10/30
27/27 - 2s - loss: 0.1942 - accuracy: 0.8657 - 2s/epoch - 75ms/step
Epoch 11/30
27/27 - 4s - loss: 0.2047 - accuracy: 0.9780 - 4s/epoch - 136ms/step
Epoch 12/30
27/27 - 3s - loss: 0.1781 - accuracy: 0.9815 - 3s/epoch - 128ms/step
Epoch 13/30
27/27 - 3s - loss: 0.1636 - accuracy: 0.98

In [ ]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,eda1,0.565051,0.555487,0.547194,0.538600
1,eda2,0.530612,0.530307,0.553571,0.552826
2,eda3,0.534439,0.531603,0.561224,0.561085
3,eda4,0.526786,0.508555,0.535714,0.533159
4,eda5,0.584184,0.582987,0.582908,0.581273


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.548214,0.541788,0.556122,0.553388
std,0.025180,0.028394,0.017651,0.019138
min,0.526786,0.508555,0.535714,0.533159
25%,0.530612,0.530307,0.547194,0.538600
50%,0.534439,0.531603,0.553571,0.552826
75%,0.565051,0.555487,0.561224,0.561085
max,0.584184,0.582987,0.582908,0.581273
